In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [2]:
csv_path = os.path.join("..", "..", "data","LINKED_DATA", "TSR_EHR", "TSR_1_CLEANED.csv")
tsr_1 = pd.read_csv(csv_path)
tsr_1.head()

,height_nm,weight_nm,edu_id,pro_id,opc_id,ih_fl,ivtpamg_nm,hospitalised_time,nivtpa_id,nivtpa1_fl,...,nihs_6br_out,nihs_7_out,nihs_8_out,nihs_9_out,nihs_10_out,nihs_11_out,total_out,SexName,Age,mrs_tx_1
0,150.0,49.0,2,1,3,0,0.0,16.0,1,0,...,0,0,0,0,0,0,8,0,66.0,4
1,153.0,62.0,3,1,3,0,0.0,8.0,0,999,...,0,1,1,0,1,0,4,0,67.0,1
2,152.0,62.0,3,1,2,0,0.0,4.0,0,999,...,0,0,1,0,0,0,1,0,69.0,1
3,148.0,56.0,2,1,2,0,0.0,5.0,0,999,...,0,0,1,0,0,0,2,0,71.0,0
4,152.0,56.0,4,1,2,0,0.0,3.0,1,0,...,0,0,0,0,0,0,0,0,71.0,0


In [3]:
tsr_1_input = tsr_1.drop(["mrs_tx_1"], axis=1)
tsr_1_input[tsr_1_input == "N"] = 0
tsr_1_input[tsr_1_input == "Y"] = 1
tsr_1_input = tsr_1_input.astype("float64")
tsr_1_input = np.array(tsr_1_input.values)

# 6 classes

In [4]:
tsr_1_output = tsr_1.mrs_tx_1
tsr_1_output = tsr_1_output.astype("float64")
tsr_1_output = np.array(tsr_1_output.values)

## SVM

In [5]:
svr = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores = cross_val_score(svr,tsr_1_input,tsr_1_output,cv = 10, scoring='r2')
print(svr_scores)
print("Mean of R^2:", svr_scores.mean())
print("Std of R^2:", svr_scores.std())

[0.74677544 0.81292754 0.82772265 0.80149462 0.82560675 0.82042926
 0.79902251 0.81432839 0.69907868 0.61754405]
Mean of R^2: 0.7764929899617632
Std of R^2: 0.0655740520079806


In [6]:
svr.fit(tsr_1_input,tsr_1_output)
svr_predict =svr.predict(tsr_1_input)
print(svr_predict)
svr_predict = np.round(svr_predict)
print(svr_predict)
print("Accuracy of RF:", (tsr_1_output == svr_predict).sum() / len(svr_predict))

[4.29422083 1.118916   0.94542222 ... 4.86915811 5.01971886 4.50174167]
[4. 1. 1. ... 5. 5. 5.]
Accuracy of RF: 0.6021632834494


In [7]:
svr_pred = cross_val_predict(svr,tsr_1_input,tsr_1_output,cv = 10)
svr_pred = np.round(svr_pred)
confusion_matrix(tsr_1_output, svr_pred)

array([[   0,    0,    0,    0,    0,    0,    0,    0],
       [   0,   37,  434,   74,   11,    1,    1,    0],
       [   1,    4,  964,  254,   36,    7,    3,    0],
       [   1,    1,  334,  458,  128,    9,    1,    0],
       [   0,    0,   26,  245,  494,   87,    3,    0],
       [   0,    0,   15,   48,  347,  866,  224,    0],
       [   0,    0,   11,   18,   36,  347, 1205,   18],
       [   0,    0,    0,    0,    0,    0,    0,    0]], dtype=int64)

In [8]:
svr_pred[svr_pred == -1] = 0
svr_pred[svr_pred == 6] = 5
confusion_matrix(tsr_1_output, svr_pred)

array([[  37,  434,   74,   11,    1,    1],
       [   5,  964,  254,   36,    7,    3],
       [   2,  334,  458,  128,    9,    1],
       [   0,   26,  245,  494,   87,    3],
       [   0,   15,   48,  347,  866,  224],
       [   0,   11,   18,   36,  347, 1223]], dtype=int64)

## RF

In [9]:
rfr = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores = cross_val_score(rfr,tsr_1_input,tsr_1_output,cv = 10, scoring='r2')
print(rfr_scores)
print("Mean of R^2:", rfr_scores.mean())
print("Std of R^2:", rfr_scores.std())

[0.80253601 0.85442144 0.84661287 0.83234409 0.854438   0.87123276
 0.82762541 0.88558175 0.78763589 0.80643935]
Mean of R^2: 0.8368867559211093
Std of R^2: 0.029832924726383228


In [10]:
rfr.fit(tsr_1_input,tsr_1_output)
rfr_predict =rfr.predict(tsr_1_input)
print(rfr_predict)
rfr_predict = np.round(rfr_predict)
print(rfr_predict)
print("Accuracy of RF:", (tsr_1_output == rfr_predict).sum() / len(rfr_predict))

[3.93333333 1.         0.93333333 ... 5.         5.         4.4       ]
[4. 1. 1. ... 5. 5. 4.]
Accuracy of RF: 0.9441398725737147


In [11]:
rfr_pred = cross_val_predict(rfr,tsr_1_input,tsr_1_output,cv = 10)
rfr_pred = np.round(rfr_pred)
confusion_matrix(tsr_1_output, rfr_pred)

array([[ 340,  172,   29,   16,    0,    1],
       [  74,  881,  258,   43,   11,    2],
       [   3,  117,  648,  146,   18,    0],
       [   0,   16,  166,  515,  151,    7],
       [   0,   14,   29,  222, 1105,  130],
       [   0,    4,   13,   31,  291, 1296]], dtype=int64)

## XGBoost

In [12]:
xgbr = XGBRegressor(n_estimators  = 15, objective="reg:squarederror", random_state = 19) 
xgbr_scores = cross_val_score(xgbr,tsr_1_input,tsr_1_output,cv = 10, scoring='r2')
print(xgbr_scores)
print("Mean of R^2:", xgbr_scores.mean())
print("Std of R^2:", xgbr_scores.std())

[0.81509538 0.85465433 0.85629336 0.84323481 0.85948638 0.87759867
 0.82663422 0.88904219 0.79003615 0.8080127 ]
Mean of R^2: 0.8420088202986703
Std of R^2: 0.029945460047586483


In [13]:
xgbr.fit(tsr_1_input,tsr_1_output)
xgbr_predict =xgbr.predict(tsr_1_input)
print(xgbr_predict)
xgbr_predict = np.round(xgbr_predict)
print(xgbr_predict)
print("Accuracy of RF:", (tsr_1_output == xgbr_predict).sum() / len(xgbr_predict))

[3.8998432  0.97976846 0.8198144  ... 4.969185   4.9114223  4.4575796 ]
[4. 1. 1. ... 5. 5. 4.]
Accuracy of RF: 0.7974514742924877


In [14]:
xgbr_pred = cross_val_predict(xgbr,tsr_1_input,tsr_1_output,cv = 10)
xgbr_pred = np.round(xgbr_pred)
confusion_matrix(tsr_1_output, xgbr_pred)

array([[ 350,  159,   34,   14,    0,    1],
       [  72,  881,  259,   46,    9,    2],
       [   3,   92,  682,  139,   16,    0],
       [   0,   14,  167,  518,  149,    7],
       [   0,   12,   36,  183, 1135,  134],
       [   0,    4,   15,   29,  290, 1297]], dtype=int64)

# 2 classes

In [15]:
tsr_1_output[(tsr_1_output == 0)|(tsr_1_output == 1)|(tsr_1_output == 2)] = 0
tsr_1_output[(tsr_1_output == 3)|(tsr_1_output == 4)|(tsr_1_output == 5)] = 1

## SVM

In [16]:
svr2 = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores2 = cross_val_score(svr2,tsr_1_input,tsr_1_output,cv = 10, scoring='r2')
print(svr_scores2)
print("Mean of R^2:", svr_scores2.mean())
print("Std of R^2:", svr_scores2.std())

[0.6497977  0.66750298 0.72486385 0.72863707 0.72866707 0.71863506
 0.68955992 0.7243377  0.50786294 0.52451408]
Mean of R^2: 0.6664378362876974
Std of R^2: 0.0795818913626931


In [17]:
svr2.fit(tsr_1_input,tsr_1_output)
svr_predict2 =svr2.predict(tsr_1_input)
print(svr_predict2)
svr_predict2 = np.round(svr_predict2)
print(svr_predict2)
print("Accuracy of RF:", (tsr_1_output == svr_predict2).sum() / len(svr_predict2))

[1.07011523 0.0224109  0.00598101 ... 0.96835053 1.13618043 0.92917206]
[1. 0. 0. ... 1. 1. 1.]
Accuracy of RF: 0.9235442287746333


In [18]:
svr_pred2 = cross_val_predict(svr2,tsr_1_input,tsr_1_output,cv = 10)
svr_pred2 = np.round(svr_pred2)
confusion_matrix(tsr_1_output, svr_pred2)

array([[2537,  222],
       [ 336, 3654]], dtype=int64)

## RF

In [19]:
rfr2 = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores2 = cross_val_score(rfr2,tsr_1_input,tsr_1_output,cv = 10, scoring='r2')
print(rfr_scores2)
print("Mean of R^2:", rfr_scores2.mean())
print("Std of R^2:", rfr_scores2.std())

[0.70414766 0.78598706 0.80979341 0.80851794 0.74737315 0.76380059
 0.76552779 0.78078116 0.65483732 0.72780284]
Mean of R^2: 0.7548568911288809
Std of R^2: 0.04593017505732098


In [20]:
rfr2.fit(tsr_1_input,tsr_1_output)
rfr_predict2 =rfr2.predict(tsr_1_input)
print(rfr_predict2)
rfr_predict2 = np.round(rfr_predict2)
print(rfr_predict2)
print("Accuracy of RF:", (tsr_1_output == rfr_predict2).sum() / len(rfr_predict2))

[1.         0.         0.         ... 1.         1.         0.93333333]
[1. 0. 0. ... 1. 1. 1.]
Accuracy of RF: 0.9964439176174248


In [21]:
rf_pred2 = cross_val_predict(rfr2,tsr_1_input,tsr_1_output,cv = 10)
rf_pred2 = np.round(rf_pred2)
confusion_matrix(tsr_1_output, rf_pred2)

array([[2495,  264],
       [ 171, 3819]], dtype=int64)

## XGBoost

In [22]:
xgbr2 = XGBRegressor(n_estimators  = 15, objective="reg:squarederror", random_state = 19) 
xgbr_scores2 = cross_val_score(xgbr2,tsr_1_input,tsr_1_output,cv = 10, scoring='r2')
print(xgbr_scores2)
print("Mean of R^2:", xgbr_scores2.mean())
print("Std of R^2:", xgbr_scores2.std())

[0.72319331 0.79790321 0.80933179 0.80348947 0.7580319  0.78636109
 0.76204846 0.81063156 0.68175723 0.76111351]
Mean of R^2: 0.769386151505116
Std of R^2: 0.03952853476472426


In [23]:
xgbr2.fit(tsr_1_input,tsr_1_output)
xgbr_predict2 =xgbr2.predict(tsr_1_input)
print(xgbr_predict2)
xgbr_predict2 = np.round(xgbr_predict2)
print(xgbr_predict2)
print("Accuracy of RF:", (tsr_1_output == xgbr_predict2).sum() / len(xgbr_predict2))

[0.9945687  0.06884833 0.01320766 ... 1.0018556  1.0013555  0.9828372 ]
[1. 0. 0. ... 1. 1. 1.]
Accuracy of RF: 0.9708104904430286


In [24]:
xgbr_pred2 = cross_val_predict(xgbr2,tsr_1_input,tsr_1_output,cv = 10)
xgbr_pred2 = np.round(xgbr_pred2)
confusion_matrix(tsr_1_output, xgbr_pred2)

array([[2501,  258],
       [ 161, 3829]], dtype=int64)

# Summary

## Mean & Std

In [25]:
svr_mean = np.array([svr_scores.mean(), svr_scores.std(), svr_scores[0], svr_scores[1], svr_scores[2], svr_scores[3],
                     svr_scores[4], svr_scores[5], svr_scores[6], svr_scores[7], svr_scores[8], svr_scores[9]])
rfr_mean = np.array([rfr_scores.mean(), rfr_scores.std(), rfr_scores[0], rfr_scores[1], rfr_scores[2], rfr_scores[3],rfr_scores[4], 
                    rfr_scores[5], rfr_scores[6], rfr_scores[7], rfr_scores[8], rfr_scores[9]])
xgbr_mean = np.array([xgbr_scores.mean(), xgbr_scores.std(), xgbr_scores[0], xgbr_scores[1], xgbr_scores[2], xgbr_scores[3],
                     xgbr_scores[4], xgbr_scores[5], xgbr_scores[6], xgbr_scores[7], xgbr_scores[8], xgbr_scores[9]])
svr_mean2 = np.array([svr_scores2.mean(), svr_scores2.std(), svr_scores2[0], svr_scores2[1], svr_scores2[2], svr_scores2[3],
                     svr_scores2[4], svr_scores2[5], svr_scores2[6], svr_scores2[7], svr_scores2[8], svr_scores2[9]])
rfr_mean2 = np.array([rfr_scores2.mean(), rfr_scores2.std(), rfr_scores2[0], rfr_scores2[1], rfr_scores2[2], rfr_scores2[3],rfr_scores2[4], 
                    rfr_scores2[5], rfr_scores2[6], rfr_scores2[7], rfr_scores2[8], rfr_scores2[9]])
xgbr_mean2 = np.array([xgbr_scores2.mean(), xgbr_scores2.std(), xgbr_scores2[0], xgbr_scores2[1], xgbr_scores2[2], xgbr_scores2[3],
                     xgbr_scores2[4], xgbr_scores2[5], xgbr_scores2[6], xgbr_scores2[7], xgbr_scores2[8], xgbr_scores2[9]])

In [26]:
tsr_1_mean = pd.DataFrame([svr_mean, rfr_mean, xgbr_mean, svr_mean2, rfr_mean2, xgbr_mean2]).T
tsr_1_mean.index = ["Mean", "Std", "R^2_1", "R^2_2", "R^2_3", "R^2_4", "R^2_5", "R^2_6", "R^2_7", "R^2_8", "R^2_9", "R^2_10"]
tsr_1_mean.columns = ["svr", 'rfr', 'xgbr', 'svr2', 'rfr2', 'xgbr2']

In [27]:
csv_save = os.path.join(".", "tsr_1_mean_regression.csv")
tsr_1_mean.to_csv(csv_save, index = True)